In [1]:
import numpy as np
import pickle
import keras
from keras import layers , activations , models , preprocessing
from keras import preprocessing , utils
# print( tf.VERSION )


Using TensorFlow backend.
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning:

## 2) Preprocessing the data

In [2]:
questions = pickle.load(open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\genericQuestions.pkl', 'rb'))
answers = pickle.load(open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\genericAnswers.pkl', 'rb'))

tokenizer = preprocessing.text.Tokenizer(filters='\t\n')
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 2326


In [2]:
# questions = pickle.load(open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\movieInput.pkl', 'rb'))
# answers = pickle.load(open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\movieOutput.pkl', 'rb'))

# size = 2000
# questions = questions[0:size]
# answers = answers[0:size]
# tokenizer = preprocessing.text.Tokenizer(filters='\t\n')
# tokenizer.fit_on_texts( questions + answers )
# VOCAB_SIZE = len( tokenizer.word_index )+1
# print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 4677


## 3) Defining the Encoder-Decoder model


In [4]:

encoder_inputs = keras.layers.Input(shape=( None , ))
encoder_embedding = keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = keras.layers.Input(shape=( None ,  ))
decoder_embedding = keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = keras.layers.Dense( VOCAB_SIZE , activation=keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    465600      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 200)    465600      input_2[0][0]                    
____________________

## 4) Training the model

In [5]:
def getOneHot(tokenized_answers, maxlen_answers):
    for i in range(len(tokenized_answers)) :
        tokenized_answers[i] = tokenized_answers[i][1:]
    padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
    onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
    return np.array( onehot_answers )
    
    
def dataGen(tokenized_questions, tokenized_answers, batchSize=10):
    maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
    padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='pre' )
    encoder_input_data = np.array( padded_questions )
    
    maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
    padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
    decoder_input_data = np.array( padded_answers )
    
    number_of_batches = len(encoder_input_data)/batchSize
    counter=0
    while(True):
        prev = batchSize*counter
        nxt = batchSize*(counter+1)
        counter+=1
        decoder_output_data = getOneHot(tokenized_answers[prev:nxt], maxlen_answers)
        yield [encoder_input_data[prev:nxt], decoder_input_data[prev:nxt]], decoder_output_data
        if counter>=number_of_batches:
            counter=0

In [6]:
batchSize = 10
epochs = 10
tokenized_questions = tokenizer.texts_to_sequences( questions )
tokenized_answers = tokenizer.texts_to_sequences( answers )
model.fit_generator(dataGen(tokenized_questions, tokenized_answers, batchSize=batchSize), 
                                      epochs=epochs, steps_per_epoch = len(tokenized_questions)/batchSize)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/10
57/56 [==============================] - 10s 172ms/step - loss: 1.2147
Epoch 2/10
57/56 [==============================] - 9s 160ms/step - loss: 1.0915
Epoch 3/10
57/56 [==============================] - 9s 153ms/step - loss: 1.0550
Epoch 4/10
57/56 [==============================] - 9s 153ms/step - loss: 1.0261
Epoch 5/10
57/56 [==============================] - 9s 161ms/step - loss: 0.9966
Epoch 6/10
57/56 [==============================] - 9s 153ms/step - loss: 0.9664
Epoch 7/10
57/56 [==============================] - 9s 150ms/step - loss: 0.9422
Epoch 8/10
57/56 [==============================] - 9s 152ms/step - loss: 0.9179
Epoch 9/10
57/56 [==============================] - 9s 151ms/step - loss: 0.8913
Epoch 10/10
57/56 [==============================] - 9s 160ms/step - loss: 0.8660


In [7]:
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )

In [1]:
import pickle
import numpy as np
import pickle
import keras
from keras import layers , activations , models , preprocessing
from keras import preprocessing , utils
def save(saveFile):
    if not os.path.isdir(saveFile):
        os.makedirs(saveFile)
    model.save(saveFile+'\\model.h5')
    metaData = {'maxlen_questions':maxlen_questions,
                'maxlen_answers':maxlen_answers}
    pickle.dump(metaData, open(saveFile+'\\metaData.pkl', 'wb'))
    pickle.dump(tokenizer, open(saveFile+'\\tokenizer.pkl', 'wb'))

def load(loadFile):
    model = keras.models.load_model(loadFile+'\\model.h5')
    metaData = pickle.load(open(loadFile+'\\metaData.pkl', 'rb'))
    tokenizer = pickle.load(open(loadFile+'\\tokenizer.pkl', 'rb'))
    return model, tokenizer, metaData['maxlen_questions'], metaData['maxlen_answers']

Using TensorFlow backend.
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Python\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning:

In [2]:
# save('testSave')
# model, tokenizer, maxlen_questions, maxlen_answers = load('testSave')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


## 5) Defining inference models

In [8]:

# def make_inference_models():
    
#     encoder_model = keras.models.Model(encoder_inputs, encoder_states)
    
#     decoder_state_input_h = keras.layers.Input(shape=( 200 ,))
#     decoder_state_input_c = keras.layers.Input(shape=( 200 ,))
    
#     decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
#     decoder_outputs, state_h, state_c = decoder_lstm(
#         decoder_embedding , initial_state=decoder_states_inputs)
#     decoder_states = [state_h, state_c]
#     decoder_outputs = decoder_dense(decoder_outputs)
#     decoder_model = keras.models.Model(
#         [decoder_inputs] + decoder_states_inputs,
#         [decoder_outputs] + decoder_states)
    
#     return encoder_model , decoder_model

# enc_model1 , dec_model1 = make_inference_models()

################################################################
def make_inference_models(model):
    
    _, stateH, stateC = model.layers[4](model.layers[2](model.inputs[0]))
    encoder = keras.models.Model(model.inputs[0], [stateH, stateC])

    inputH = keras.layers.Input(shape=(200,), name='inpH')
    inputC = keras.layers.Input(shape=(200,), name='inpC')
    

    decoderOut, stateH2, stateC2 = model.layers[5](model.layers[3](model.inputs[-1]), 
                                                   initial_state=[inputH, inputC])
        
    decoder = keras.models.Model([model.inputs[-1]] + [inputH, inputC], 
                               [model.layers[-1](decoderOut)] + [stateH2, stateC2])
    
    return encoder , decoder

enc_model2 , dec_model2 = make_inference_models(model)

## 6) Talking with our Chatbot


In [9]:
def str_to_tokens( sentence : str , maxlen_questions):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='pre')

In [13]:
for sentence in questions[0:10]:
    print(sentence)
    padSentence = preprocessing.sequence.pad_sequences([tokenizer.texts_to_sequences([sentence])[0]] , 
                                                       maxlen=maxlen_questions , padding='pre')
    states_values = enc_model2.predict(padSentence)
#     states_values = enc_model2.predict( str_to_tokens(sentence, maxlen_questions) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['<start>']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model2.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == '<end>' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

What is AI?
 is the the science of science of production of the high of society. of production of the high of various phase. of various society. attack hypothetical bioinformatics sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun
What is AI?
 is the the science of science of production of the high of society. of production of the high of various phase. of various society. attack hypothetical bioinformatics sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun
Are you sentient?
 i am a lot <end>
Are you sentient?
 i am a lot <end>
Are you sentient?
 i am a lot <end>
Are you sapient?
 i am a lot <end>
Are you sapient?
 i am a lot <end>
Are you sapient?
 i am a lot <end>
Are you sapient?
 i am a lot <end>
What lan

In [26]:
for sentence in questions[0:10]:
    print(sentence)
    padSentence = preprocessing.sequence.pad_sequences([tokenizer.texts_to_sequences([sentence])[0]] , 
                                                       maxlen=maxlen_questions , padding='pre')
    statesValues = enc_model2.predict(padSentence)
    inpTargetSeq = np.zeros( ( 1 , 1 ) )
    inpTargetSeq[0, 0] = tokenizer.word_index['<start>']
    reply = ''
    while (1):
        decOut , h , c = dec_model2.predict([ inpTargetSeq ] + statesValues )
        predIndex = np.argmax(decOut[0][0])
        predWord = tokenizer.index_word[predIndex]
        reply += ' {}'.format(predWord)
        
        if predWord == '<end>' or len(reply.split()) > maxlen_answers:
            break
            
        inpTargetSeq[ 0 , 0 ] = predIndex
        statesValues = [ h , c ] 

    print( reply )

What is AI?
 is the the science of science of production of the high of society. of production of the high of various phase. of various society. attack hypothetical bioinformatics sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun
What is AI?
 is the the science of science of production of the high of society. of production of the high of various phase. of various society. attack hypothetical bioinformatics sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun sun
Are you sentient?
 i am a lot <end>
Are you sentient?
 i am a lot <end>
Are you sentient?
 i am a lot <end>
Are you sapient?
 i am a lot <end>
Are you sapient?
 i am a lot <end>
Are you sapient?
 i am a lot <end>
Are you sapient?
 i am a lot <end>
What lan

In [12]:
dec_outputs[0, -1, :]

array([1.5109004e-03, 2.1564250e-09, 3.7139004e-01, ..., 3.6409412e-05,
       3.9359611e-06, 2.0602038e-06], dtype=float32)

# Attention

In [3]:
import tensorflow as tf
import os
from tensorflow.python.keras.layers import Layer
from tensorflow.python.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state """

            assert_msg = "States must be a list. However states {} is of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch_size*en_seq_len, latent_dim
            reshaped_enc_outputs = K.reshape(encoder_out_seq, (-1, en_hidden))
            # <= batch_size*en_seq_len, latent_dim
            W_a_dot_s = K.reshape(K.dot(reshaped_enc_outputs, self.W_a), (-1, en_seq_len, en_hidden))
            if verbose:
                print('wa.s>',W_a_dot_s.shape)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>',U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            reshaped_Ws_plus_Uh = K.tanh(K.reshape(W_a_dot_s + U_a_dot_h, (-1, en_hidden)))
            if verbose:
                print('Ws+Uh>', reshaped_Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.reshape(K.dot(reshaped_Ws_plus_Uh, self.V_a), (-1, en_seq_len))
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """
            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        def create_inital_state(inputs, hidden_size):
            # We are not using initial states, but need to pass something to K.rnn funciton
            fake_state = K.zeros_like(inputs)  # <= (batch_size, enc_seq_len, latent_dim
            fake_state = K.sum(fake_state, axis=[1, 2])  # <= (batch_size)
            fake_state = K.expand_dims(fake_state)  # <= (batch_size, 1)
            fake_state = K.tile(fake_state, [1, hidden_size])  # <= (batch_size, latent_dim
            return fake_state

        fake_state_c = create_inital_state(encoder_out_seq, encoder_out_seq.shape[-1])
        fake_state_e = create_inital_state(encoder_out_seq, encoder_out_seq.shape[1])  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [4]:
from tensorflow.python.keras.layers import Input, GRU, Dense, Concatenate, TimeDistributed
from tensorflow.python.keras.models import Model


def define_nmt(hidden_size, batch_size, en_timesteps, en_vsize, fr_timesteps, fr_vsize):
    """ Defining a NMT model """

    # Define an input sequence and process it.
    if batch_size:
        encoder_inputs = Input(batch_shape=(batch_size, en_timesteps, en_vsize), name='encoder_inputs')
        decoder_inputs = Input(batch_shape=(batch_size, fr_timesteps - 1, fr_vsize), name='decoder_inputs')
    else:
        encoder_inputs = Input(shape=(en_timesteps, en_vsize), name='encoder_inputs')
        decoder_inputs = Input(shape=(fr_timesteps - 1, fr_vsize), name='decoder_inputs')

    # Encoder GRU
    encoder_gru = GRU(hidden_size, return_sequences=True, return_state=True, name='encoder_gru')
    encoder_out, encoder_state = encoder_gru(encoder_inputs)

    # Set up the decoder GRU, using `encoder_states` as initial state.
    decoder_gru = GRU(hidden_size, return_sequences=True, return_state=True, name='decoder_gru')
    decoder_out, decoder_state = decoder_gru(decoder_inputs, initial_state=encoder_state)

    # Attention layer
    attn_layer = AttentionLayer(name='attention_layer')
    attn_out, attn_states = attn_layer([encoder_out, decoder_out])

    # Concat attention input and decoder GRU output
    decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_out, attn_out])

    # Dense layer
    dense = Dense(fr_vsize, activation='softmax', name='softmax_layer')
    dense_time = TimeDistributed(dense, name='time_distributed_layer')
    decoder_pred = dense_time(decoder_concat_input)

    # Full model
    full_model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_pred)
    full_model.compile(optimizer='adam', loss='categorical_crossentropy')

    full_model.summary()

    """ Inference model """
    batch_size = 1

    """ Encoder (Inference) model """
    encoder_inf_inputs = Input(batch_shape=(batch_size, en_timesteps, en_vsize), name='encoder_inf_inputs')
    encoder_inf_out, encoder_inf_state = encoder_gru(encoder_inf_inputs)
    encoder_model = Model(inputs=encoder_inf_inputs, outputs=[encoder_inf_out, encoder_inf_state])

    """ Decoder (Inference) model """
    decoder_inf_inputs = Input(batch_shape=(batch_size, 1, fr_vsize), name='decoder_word_inputs')
    encoder_inf_states = Input(batch_shape=(batch_size, en_timesteps, hidden_size), name='encoder_inf_states')
    decoder_init_state = Input(batch_shape=(batch_size, hidden_size), name='decoder_init')

    decoder_inf_out, decoder_inf_state = decoder_gru(decoder_inf_inputs, initial_state=decoder_init_state)
    attn_inf_out, attn_inf_states = attn_layer([encoder_inf_states, decoder_inf_out])
    decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_inf_out, attn_inf_out])
    decoder_inf_pred = TimeDistributed(dense)(decoder_inf_concat)
    decoder_model = Model(inputs=[encoder_inf_states, decoder_init_state, decoder_inf_inputs],
                          outputs=[decoder_inf_pred, attn_inf_states, decoder_inf_state])

    return full_model, encoder_model, decoder_model

def sents2sequences(tokenizer, sentences, reverse=False, pad_length=None, padding_type='post'):
    encoded_text = tokenizer.texts_to_sequences(sentences)
    preproc_text = preprocessing.sequence.pad_sequences(encoded_text, padding=padding_type, maxlen=pad_length)
    if reverse:
        preproc_text = np.flip(preproc_text, axis=1)

    return preproc_text

In [5]:
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.keras.backend import set_session
tf.keras.backend.clear_session()  # For easy reset of notebook state.

config_proto = tf.ConfigProto()
off = rewriter_config_pb2.RewriterConfig.OFF
config_proto.graph_options.rewrite_options.arithmetic_optimization = off
session = tf.Session(config=config_proto)
set_session(session)

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(filters='\t\n', oov_token='UNK')
tokenizer.fit_on_texts(questions + answers)

tokenized_questions = tokenizer.texts_to_sequences( questions )
tokenized_answers = tokenizer.texts_to_sequences( answers )

maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )

encoder_input_data = sents2sequences(tokenizer, questions, reverse=False, padding_type='pre', pad_length=maxlen_questions)
decoder_input_data = sents2sequences(tokenizer, answers, pad_length=maxlen_answers)

# padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
# encoder_input_data = np.array( padded_questions )

# padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
# decoder_input_data = np.array( padded_answers )

vsize = max(tokenizer.index_word.keys()) + 1

In [7]:
full_model, infer_enc_model, infer_dec_model = define_nmt(
        hidden_size=96, batch_size=64,
        en_timesteps=maxlen_questions, fr_timesteps=maxlen_answers,
        en_vsize=vsize, fr_vsize=vsize)

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     (64, 22, 2329)       0                                            
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     (64, 73, 2329)       0                                            
__________________________________________________________________________________________________
encoder_gru (GRU)               [(64, 22, 96), (64,  698688      encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_gru (GRU)               [(64, 73, 96), (64,  698688      decoder_inputs[0][0]             
                                     

In [29]:
fr_onehot_seq[0]

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [8]:
n_epochs = 10
batch_size =64
for ep in range(n_epochs):
    losses = []
    for bi in range(0, encoder_input_data.shape[0] - batch_size, batch_size):

        en_onehot_seq = utils.to_categorical(encoder_input_data[bi:bi + batch_size, :], num_classes=vsize)
        fr_onehot_seq = utils.to_categorical(decoder_input_data[bi:bi + batch_size, :], num_classes=vsize)

        full_model.train_on_batch([en_onehot_seq, fr_onehot_seq[:, :-1, :]], fr_onehot_seq[:, 1:, :])

        l = full_model.evaluate([en_onehot_seq, fr_onehot_seq[:, :-1, :]], fr_onehot_seq[:, 1:, :],
                                batch_size=batch_size, verbose=0)

        losses.append(l)
    if (ep + 1) % 1 == 0:
        print("Loss in epoch {}: {}".format(ep + 1, np.mean(losses)))

Instructions for updating:
Use tf.cast instead.
Loss in epoch 1: 7.637436032295227
Loss in epoch 2: 7.2712976932525635
Loss in epoch 3: 4.909414708614349
Loss in epoch 4: 1.5088359713554382
Loss in epoch 5: 1.5009416937828064
Loss in epoch 6: 1.3769365176558495
Loss in epoch 7: 1.3488154262304306
Loss in epoch 8: 1.3201587498188019
Loss in epoch 9: 1.2837055698037148
Loss in epoch 10: 1.2385107427835464


In [27]:
for sentences in questions[0:10]:
    print(sentences)
    test_en_seq = sents2sequences(tokenizer, [sentences], pad_length=maxlen_questions)
    test_fr_seq = sents2sequences(tokenizer, ['<start>'], vsize)
    test_en_onehot_seq = utils.to_categorical(test_en_seq, num_classes=vsize)
    test_fr_onehot_seq = np.expand_dims(utils.to_categorical(test_fr_seq, num_classes=vsize), 1)

    enc_outs, enc_last_state = infer_enc_model.predict(test_en_onehot_seq)
    dec_state = enc_last_state
    attention_weights = []
    fr_text = ''
    for i in range(20):

        dec_out, attention, dec_state = infer_dec_model.predict([enc_outs, dec_state, test_fr_onehot_seq])
        dec_ind = np.argmax(dec_out, axis=-1)[0, 0]

        if dec_ind == 0:
            break
        fr_index2word = dict(zip(fr_tokenizer.word_index.values(), fr_tokenizer.word_index.keys()))
        test_fr_seq = sents2sequences(fr_tokenizer, [fr_index2word[dec_ind]], vsize)
        test_fr_onehot_seq = np.expand_dims(utils.to_categorical(test_fr_seq, num_classes=vsize), 1)

        attention_weights.append((dec_ind, attention))
        fr_text += fr_index2word[dec_ind] + ' '
    print(fr_text)

What is AI?

What is AI?

Are you sentient?

Are you sentient?

Are you sentient?

Are you sapient?

Are you sapient?

Are you sapient?

Are you sapient?

What language are you written in?



In [28]:
dec_ind

0